In [2]:
#initial imports that you may find useful
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
import numpy as np

For this assignment you will need to use the sklearn framework to implement a custom Naive Bayes classifier.  The classifier only needs to handle binary data (both the attributes and the classes).  The attributes will always have a value of 0 or 1.  The class labels will always have a value of 1 or -1.  You can use libraries to help with the data processing, calculations, etc, but you must implement your own Naïve Bayes algorithm.  Do not use an existing implementation.  One important implementation detail is that you should convert the probabilities to log probabilities to avoid the number becoming to small to represent as a floating point number.  For example instead of computing P(x|c)P(c) compute log(P(x|c)+log(P(c)). Provide your implementation below.

In [3]:

#Your Naive Bayes Implementation goes here.
#Adjust this as you see fit

class BinaryNBClassifier(BaseEstimator, ClassifierMixin):        
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        return self

    def train(self, training_set):

        # count spam and non-spam messages
        num_spams = len([is_spam
                         for message, is_spam in training_set
                         if is_spam])
        num_non_spams = len(training_set) - num_spams

        # run training data through our "pipeline"
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts,
                                             num_spams,
                                             num_non_spams,
                                             self.k)

    def classify(self, message):
        return spam_probability(self.word_probs, message)
        
    def predict(self, X):
        return np.random.randint(0, self.classes_.size,
                                 size=X.shape[0])
    
    pass
    
def tokenize(message):
    message = message.lower()                       # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message)   # extract the words
    return set(all_words)                           # remove duplicates


def count_words(training_set):
    """training set consists of pairs (message, is_spam)"""
    counts = defaultdict(lambda: [0, 0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1
    return counts

def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """turn the word_counts into a list of triplets
    w, p(w | spam) and p(w | ~spam)"""
    return [(w,
            (spam + k) / (total_spams + 2 * k),
            (non_spam + k) / (total_non_spams + 2 * k))
            for w, (spam, non_spam) in counts.iteritems()]

def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0

    # iterate through each word in our vocabulary
    for word, prob_if_spam, prob_if_not_spam in word_probs:

        # if *word* appears in the message,
        # add the log probability of seeing it
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)

        # if *word* doesn't appear in the message
        # add the log probability of _not_ seeing it
        # which is log(1 - probability of seeing it)
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)

    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)



seed = np.random
# random.seed(0)      # just so you get the same answers as me
train_data, test_data = split_data(data, 0.75)

classifier = NaiveBayesClassifier()
classifier.train(train_data)

NameError: name 'split_data' is not defined

Helpful Resources: 
* https://sklearn-template.readthedocs.io/en/latest/user_guide.html
* https://saturncloud.io/blog/how-to-keep-column-names-when-converting-from-pandas-to-numpy
*

Now you will train and test your Binary Naive Bayes classifier on a few different datasets.  The datasets can be downloaded from canvas.  They are linked in the assignment description.  For this part of the assignment we will not be splitting the data into training, validation and test data sets.  Instead you should use the entire dataset for training and the entire dataset for testing.  You will need to complete the following table (you can just output the results in this format you don't need to copy them into the text field).

|dataset|# of instances|# of features | Your NB Training Time | Your NB Test Time | Your NB Accuracy | sklearn CategoricalNB Training Time | sklearn Categorical NB Test Time | sklearn CategoricalNB Accuracy|
|-----------|------------|-------------|------------------|-------------------|-------------------------|---------------------------------|------------------------|----------------------------------|
test1_1 |
test1_2 |
test1_4 |
test1_5 |
test2_1 |
test2_2 |
test2_4 |
test2_5 |
test4_1 |
test4_2 |
test4_4 |
test4_5 |
test5_1 |
test5_2 |
test5_4 |


In [3]:
#Train and test your BinaryNBClassifier and the sklearn CategoricalNBClassifier on the datasets from canvas


The next step for this assignment is split the vote dataset (also found on canvas) into a *train/test split* (use 20% of the data for testing).  Train both algorithms on the training data using *cross-fold validation* and then report the accuracy, f1-score, mcc and informedness results.

In [4]:
#Split the vote dataset
#Use cross-validatation to compare BinaryNBClassifier against CategoricalNBClassifier

Finally, choose the algorithm that performed the best on the cross-validation, train it on all the training data and test on the test data.  Report the accuracy, f1-score, mcc and informedness results

In [5]:
#Final Generalization test